In [1]:
# Run While Mounting a Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.0 MB/s eta 0:00:00


In [3]:
#boundry box updated code

import cv2
import numpy as np
from ultralytics import YOLO

# --- 1. SETUP ---
# Load the pre-trained YOLOv8 model
model = YOLO('/content/drive/MyDrive/AI_based_Intellegent_Traffic_Monitoring_and_Management_System/runs/detect/train2/weights/best.pt')

# --- 2. EDIT THESE ---
# Paste the exact path to YOUR video file
input_video_path = '/content/drive/MyDrive/AI_based_Intellegent_Traffic_Monitoring_and_Management_System/Data/emergency_vehical.mp4'

# Define the coordinates of YOUR lane's polygon
ROI_POINTS = np.array([(150, 100), (1300, 150), (1300, 600), (150, 600)], np.int32)
# --- END OF EDIT SECTION ---

# Define the output path
output_video_path = 'output_vehicle_count.mp4'

# COCO class IDs for vehicles (car, motorcycle, bus, truck)
VEHICLE_CLASS_IDS = [2, 3, 5, 7]

# --- 3. VIDEO PROCESSING ---
cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

print("Processing video... This may take a moment.")

# --- Inside your 01_Main_Vehicle_Counter.ipynb ---

try:
    while True:
        success, frame = cap.read()
        if not success:
            break

        # --- Initialize TWO counters for each frame ---
        normal_vehicle_count = 0
        emergency_vehicle_count = 0

        # Draw the ROI polygon
        cv2.polylines(frame, [ROI_POINTS], isClosed=True, color=(255, 0, 0), thickness=2)

        # Run YOLO detection
        results = model(frame)

        # Define your emergency vehicle classes
        EMERGENCY_VEHICLE_CLASSES = ['ambulance', 'fire_truck', 'police_car']

        for box in results[0].boxes:
            cls_id = int(box.cls)
            label = model.names[cls_id]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

            # Check if the vehicle is inside the ROI
            if cv2.pointPolygonTest(ROI_POINTS, (center_x, center_y), False) >= 0:

                # Check if it's an emergency vehicle and increment the correct counter
                if label in EMERGENCY_VEHICLE_CLASSES:
                    emergency_vehicle_count += 1
                    color = (0, 0, 255) # Red for emergency
                else:
                    normal_vehicle_count += 1
                    color = (0, 255, 0) # Green for normal

                # Draw the bounding box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # --- Display BOTH counts on the screen ---
        # Display normal vehicle count in white
        cv2.putText(frame, f"Normal Vehicles: {normal_vehicle_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        # Display emergency vehicle count in red below the first count
        cv2.putText(frame, f"Emergency Vehicles: {emergency_vehicle_count}", (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        out.write(frame)

finally:
    print("Processing complete. Releasing resources.")
    cap.release()
    out.release()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Processing video... This may take a moment.

0: 256x416 (no detections), 222.7ms
Speed: 13.9ms preprocess, 222.7ms inference, 14.8ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 66.9ms
Speed: 2.0ms preprocess, 66.9ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 60.4ms
Speed: 2.5ms preprocess, 60.4ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 67.7ms
Speed: 2.3ms preprocess, 67.7ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 61.0ms
Speed: 1.8ms preprocess, 61.0ms inference, 0.7ms postprocess per image at